In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [8]:
page = requests.get("https://weather.com/zh-TW/weather/tenday/l/TWXX0019:1:TW#!")


In [9]:
soup = BeautifulSoup(page.content, 'html.parser')
seven_day = soup.find(id="twc-scrollabe")
forecast_items = seven_day.find_all(class_="twc-sticky-col")


In [59]:
def cleaner(data):
    for loo in data:
        if len(loo) < 3:
            
            # scraping current temperature
            temp_now = soup.find(id="APP")
            loo[0] = temp_now.find(class_="styles-xz0ANuUJ__temperature__3Ph8k").get_text().replace("°","")
            
            # cleaning "--" and substract integer
            loo[1] = ''.join(filter(str.isdigit, loo[0]))
            
        else:
             del loo[-1] # drop the last empty element
    return data

In [60]:
period_tags = seven_day.select(".twc-sticky-col .day-detail")
periods = [pt.get_text() for pt in period_tags]
periods_clean = [ptt.replace("日", "").split("月 ") for ptt in periods]

# temperature 
short_descs = [sd.get_text() for sd in seven_day.select(".temp")]
descs = [ptt.split("°") for ptt in short_descs][1::]
descs = cleaner(descs)
high = [tt[0] for tt in descs]
low = [tt[1] for tt in descs]

precip = [t.get_text() for t in seven_day.select(".precip")][1::]
humidity = [t.get_text() for t in seven_day.select(".humidity")][1::]

In [78]:
# popoff = lambda x: x.pop(0)

import datetime
today = [datetime.date.today()]
one_day = datetime.timedelta(days=1)

for days in range(0,len(precip)-1):
    today.append(today[0] + (days + 1) * one_day)


In [89]:
weather = pd.DataFrame({
        "Temp_higest": high, 
        "Temp_lowest": low, 
        "precip": precip,
        "humidity": humidity
    }, index = today)

In [91]:
weather

,Temp_higest,Temp_lowest,humidity,precip
2018-05-24,32,24,59%,30%
2018-05-25,33,24,62%,40%
2018-05-26,34,24,56%,0%
2018-05-27,35,25,56%,0%
2018-05-28,34,25,59%,10%
2018-05-29,34,25,61%,10%
2018-05-30,33,25,60%,20%
2018-05-31,33,25,60%,20%
2018-06-01,32,24,62%,40%
2018-06-02,31,24,62%,60%


In [12]:
temp_nums = weather["temp"].str.extract("(?P<temp_num>\d+)", expand=False)
weather["temp_num"] = temp_nums.astype('int')

is_night = weather["temp"].str.contains("Low")
weather["is_night"] = is_night


In [13]:
weather[is_night]

,desc,period,short_desc,temp,temp_num,is_night
0,Tonight: Patchy drizzle after 2am. Increasing...,Tonight,Mostly Cloudythen PatchyDrizzle,Low: 53 °F,53,True
2,"Tuesday Night: Partly cloudy, with a low aroun...",TuesdayNight,Partly Cloudy,Low: 52 °F,52,True
4,"Wednesday Night: Partly cloudy, with a low aro...",WednesdayNight,Partly Cloudy,Low: 52 °F,52,True
6,"Thursday Night: Partly cloudy, with a low arou...",ThursdayNight,Partly Cloudyand Breezythen MostlyCloudy,Low: 52 °F,52,True
8,"Friday Night: Mostly cloudy, with a low around...",FridayNight,Mostly Cloudy,Low: 52 °F,52,True
